In [1]:
import os
import os.path
import sys
from pathlib import Path
sys.path.insert(0, '..')
from elitetools import eddb
eddb.load_feeds()

# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-commodities.json".
# Feeds.COMMODITIES records loaded:  373
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-listings.csv".
# Feeds.PRICES records loaded:  4191028
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-factions.jsonl".
# Feeds.FACTIONS records loaded:  77156
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-systems_populated.jsonl".
# Feeds.POPULATED_SYSTEMS records loaded:  20568
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-stations.jsonl".
# Feeds.STATIONS records loaded:  76104
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\scraped_listings.jsonl".
# Real-time listing records loaded:  325


In [105]:
origin_name="Jimavi"
commodity_names=['Bioreducing Lichen', 'Agronomic Treatment', 'Consumer Technology']
minimum_supply = 100

In [106]:
origin = eddb.find_system_by_name(origin_name)
commodities = eddb.commodity_details[eddb.commodity_details['name'].isin(commodity_names)][['id', 'name']] \
    .rename(columns={'name': 'commodity', 'id': 'commodity_id'})
commodities

,commodity_id,commodity
5,6,Consumer Technology
64,65,Bioreducing Lichen
356,360,Agronomic Treatment


In [107]:
listings = eddb.commodity_listings_eod.merge(commodities, on='commodity_id') \
    [['station_id', 'commodity', 'supply', 'buy_price']] \
    .query(f'supply > {minimum_supply}') \
    .drop_duplicates()
listings

,station_id,commodity,supply,buy_price
1,3,Consumer Technology,97213,6144
12,14,Consumer Technology,62658,5988
13,15,Consumer Technology,68434,6287
25,27,Consumer Technology,124485,5822
28,30,Consumer Technology,32477,6116
...,...,...,...,...
130987,71063,Bioreducing Lichen,3702,992
130988,71070,Bioreducing Lichen,16733,1000
130990,71087,Bioreducing Lichen,973,910
130992,71472,Bioreducing Lichen,1292,650


In [108]:
pos_listings = listings.merge(eddb.station_details[['id', 'name', 'system_id', 'max_landing_pad_size', 'distance_to_star']] \
    .rename(columns={'id': 'station_id', 'name': 'station', 'max_landing_pad_size': 'Pad Size', 'distance_to_star': 'Sublight'}), on='station_id') \
    .merge(eddb.populated_systems[['id', 'name', 'x', 'y', 'z']] \
    .rename(columns={'id': 'system_id', 'name': 'system'}), on='system_id') \
    .drop(columns=['station_id', 'system_id'])

pos_listings

,commodity,supply,buy_price,station,Pad Size,Sublight,system,x,y,z
0,Consumer Technology,97213,6144,Gehry Dock,M,3816.0,HIP 29444,147.37500,-80.46875,13.56250
1,Agronomic Treatment,3009,9165,Gehry Dock,M,3816.0,HIP 29444,147.37500,-80.46875,13.56250
2,Bioreducing Lichen,9054,1053,Gehry Dock,M,3816.0,HIP 29444,147.37500,-80.46875,13.56250
3,Consumer Technology,117386,5861,Swift Vision,L,2768.0,HIP 29444,147.37500,-80.46875,13.56250
4,Agronomic Treatment,6312,4669,Swift Vision,L,2768.0,HIP 29444,147.37500,-80.46875,13.56250
...,...,...,...,...,...,...,...,...,...,...
4430,Agronomic Treatment,992,2004,X3B-T3M,L,1605.0,Exioce,78.50000,-100.25000,16.78125
4431,Agronomic Treatment,636,6335,V0L-7XT,L,1165.0,Lalande 4268,-13.81250,-64.12500,-44.84375
4432,Agronomic Treatment,1304,158,XFB-GKW,L,433.0,Alanaribri,-32.84375,-75.56250,-94.06250
4433,Bioreducing Lichen,881,51,XFB-GKW,L,433.0,Alanaribri,-32.84375,-75.56250,-94.06250


In [109]:
dist_listings = pos_listings.assign(distance = lambda x: eddb.distance(origin, x))
dist_listings

,commodity,supply,buy_price,station,Pad Size,Sublight,system,x,y,z,distance
0,Consumer Technology,97213,6144,Gehry Dock,M,3816.0,HIP 29444,147.37500,-80.46875,13.56250,67.8
1,Agronomic Treatment,3009,9165,Gehry Dock,M,3816.0,HIP 29444,147.37500,-80.46875,13.56250,67.8
2,Bioreducing Lichen,9054,1053,Gehry Dock,M,3816.0,HIP 29444,147.37500,-80.46875,13.56250,67.8
3,Consumer Technology,117386,5861,Swift Vision,L,2768.0,HIP 29444,147.37500,-80.46875,13.56250,67.8
4,Agronomic Treatment,6312,4669,Swift Vision,L,2768.0,HIP 29444,147.37500,-80.46875,13.56250,67.8
...,...,...,...,...,...,...,...,...,...,...,...
4430,Agronomic Treatment,992,2004,X3B-T3M,L,1605.0,Exioce,78.50000,-100.25000,16.78125,14.7
4431,Agronomic Treatment,636,6335,V0L-7XT,L,1165.0,Lalande 4268,-13.81250,-64.12500,-44.84375,112.8
4432,Agronomic Treatment,1304,158,XFB-GKW,L,433.0,Alanaribri,-32.84375,-75.56250,-94.06250,156.6
4433,Bioreducing Lichen,881,51,XFB-GKW,L,433.0,Alanaribri,-32.84375,-75.56250,-94.06250,156.6


In [110]:
top_count = 5
listings_ranked = dist_listings.assign(rnk = dist_listings.groupby('commodity')['distance'] \
        .rank(method='first', ascending=True)) \
        .query(f'rnk <= {top_count}') \
        .sort_values(by=['distance', 'commodity', 'station'], ascending=[True, True, True]) \
        .drop_duplicates() \
        .reset_index() \
        .drop(columns=['x', 'y', 'z', 'index', 'rnk'])
listings_ranked

,commodity,supply,buy_price,station,Pad Size,Sublight,system,distance
0,Agronomic Treatment,15401,4634,Chernykh Ring,L,30.0,Dvorsi,13.0
1,Agronomic Treatment,9148,9279,Fowler Orbital,L,18.0,Dvorsi,13.0
2,Agronomic Treatment,5569,2562,Qureshi Vision,M,74654.0,Dvorsi,13.0
3,Bioreducing Lichen,12143,860,Chernykh Ring,L,30.0,Dvorsi,13.0
4,Bioreducing Lichen,354482,761,Fowler Orbital,L,18.0,Dvorsi,13.0
5,Bioreducing Lichen,29067,848,Qureshi Vision,M,74654.0,Dvorsi,13.0
6,Consumer Technology,110878,5845,Fowler Orbital,L,18.0,Dvorsi,13.0
7,Consumer Technology,12285,6224,Qureshi Vision,M,74654.0,Dvorsi,13.0
8,Agronomic Treatment,8924,5183,Andoyer Station,L,61.0,Gongalungul,13.4
9,Agronomic Treatment,5518,4711,van der Riet Woolley Ring,M,61.0,Gongalungul,13.4


In [111]:
listings_grouped = listings_ranked.sort_values(by=['distance', 'system', 'Sublight', 'station', 'commodity']) \
    .set_index(['system', 'station', 'commodity'])
listings_grouped

supply  buy_price  \
system      station                   commodity                                
Dvorsi      Fowler Orbital            Agronomic Treatment    9148       9279   
                                      Bioreducing Lichen   354482        761   
                                      Consumer Technology  110878       5845   
            Chernykh Ring             Agronomic Treatment   15401       4634   
                                      Bioreducing Lichen    12143        860   
            Qureshi Vision            Agronomic Treatment    5569       2562   
                                      Bioreducing Lichen    29067        848   
                                      Consumer Technology   12285       6224   
Gongalungul Andoyer Station           Agronomic Treatment    8924       5183   
            van der Riet Woolley Ring Agronomic Treatment    5518       4711   
Ankou       Kirby Orbital             Bioreducing Lichen      554        919   
Calhuacan   Sakers Holdings           Bioreducing Lichen    22603        723   
                                      Consumer Technology    5392       5719   
Njiri       Helin Dock                Consumer Technology   30767       6096   
            Grant Terminal            Consumer Technology   14407       6202   

                                                          Pad Size  Sublight  \
system      station                   commodity                                
Dvorsi      Fowler Orbital            Agronomic Treatment        L      18.0   
                                      Bioreducing Lichen         L      18.0   
                                      Consumer Technology        L      18.0   
            Chernykh Ring             Agronomic Treatment        L      30.0   
                                      Bioreducing Lichen         L      30.0   
            Qureshi Vision            Agronomic Treatment        M   74654.0   
                                      Bioreducing Lichen         M   74654.0   
                                      Consumer Technology        M   74654.0   
Gongalungul Andoyer Station           Agronomic Treatment        L      61.0   
            van der Riet Woolley Ring Agronomic Treatment        M      61.0   
Ankou       Kirby Orbital             Bioreducing Lichen         L   40311.0   
Calhuacan   Sakers Holdings           Bioreducing Lichen         L     395.0   
                                      Consumer Technology        L     395.0   
Njiri       Helin Dock                Consumer Technology        L     151.0   
            Grant Terminal            Consumer Technology        M     191.0   

                                                           distance  
system      station                   commodity                      
Dvorsi      Fowler Orbital            Agronomic Treatment      13.0  
                                      Bioreducing Lichen       13.0  
                                      Consumer Technology      13.0  
            Chernykh Ring             Agronomic Treatment      13.0  
                                      Bioreducing Lichen       13.0  
            Qureshi Vision            Agronomic Treatment      13.0  
                                      Bioreducing Lichen       13.0  
                                      Consumer Technology      13.0  
Gongalungul Andoyer Station           Agronomic Treatment      13.4  
            van der Riet Woolley Ring Agronomic Treatment      13.4  
Ankou       Kirby Orbital             Bioreducing Lichen       16.1  
Calhuacan   Sakers Holdings           Bioreducing Lichen       16.4  
                                      Consumer Technology      16.4  
Njiri       Helin Dock                Consumer Technology      19.1  
            Grant Terminal            Consumer Technology      19.1